In [11]:
import numpy as np


def img_to_col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    行列演算の最適化のために次元をつぶす
    (平面を直線にする)
    
    Parameters
    ----------
    input_date: (データ数, チャンネル, 高さ, 幅) バッチデータ
    filter_h: フィルターの高さ
    filter_w: フィルターの幅
    stride: ストライド
    pad: パディング
    """
    N, C, H, W = input_data.shape
    out_h = (H+2*pad - filter_h) // stride + 1
    out_w = (W+2*pad - filter_w) // stride + 1
    
    # 各データにパディングをつけてる
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:,:,y,x,:,:] = img[:,:,y:y_max:stride,x:x_max:stride]
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [12]:
from common.util import col2im
class Convolution:
    """
    Attributes
    ----------
    畳み込み層
    """
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（バックプロパゲーションで使う)
        self.x = None
        self.col = None
        self.col_W = None

        # 重み・バイアスの勾配
        self.dW = None
        self.db = None
        
    def forward(self, x):
        """
        推論処理
        """
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int(H+2*self.pad - FH/self.stride)
        out_w = 1 + int(W+2*self.pad - FW/self.stride)
        
        col = img_to_col(x, FH, FW, self.stride, self.pad)
        # フィルターを一行ずつ並べた行列の転置行列
        col_W = self.W.reshape(FN, -1).T
        
        # 順方向にデータを流す
        out = np.dot(col, col_W) + self.b
        # この変換で最終的なデータの形は(N, -1(自動推定), out_h, out_w)になる
        out= self.W.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        self.x = x
        self.col = col
        self.col_W = col_W

        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)
        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
        
        dcol = np.dot(dout, self.col_W.T, dout)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        
        return dx

In [20]:
"""
畳み込み層を使ったニューラルネットワークの実装
"""

import numpy as np
import pickle
from collections import OrderedDict
from common.layers import Relu, Pooling, Affine, Dropout, SoftmaxWithLoss

class DeepConvNet:
    """
    認識率99%以上の高精度ConvNet
    
    ネットワークの構成
    conv -> relu -> conv -> relu -> pool ->
    conv -> relu -> conv -> relu -> pool ->
    conv -> relu -> conv -> relu -> pool ->
    affine -> relu -> dropout -> affine -> dropout -> softmax
    """
    def __init__(
            self, 
            input_dim=(1, 28, 28),
            conv_param_1 = {'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1},
            conv_param_2 = {'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1},
            conv_param_3 = {'filter_num': 32, 'filter_size': 3, 'pad': 1, 'stride': 1},
            conv_param_4 = {'filter_num': 32, 'filter_size': 3, 'pad': 2, 'stride': 1},
            conv_param_5 = {'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1},
            conv_param_6 = {'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1},
            hidden_size=50, 
            outout_size=10
    ):
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        # reluを使う場合に推奨される初期値
        weight_init_scales = np.sqrt(2.0/pre_node_nums)

        self.params ={}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W'+str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], conv_param['filter_size'])
            self.params['b'+str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
            
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size) 
        self.params['W8'] = weight_init_scales[6] * np.random.randn(hidden_size, outout_size)
        self.params['b8'] = np.zeros(outout_size) 
        
        # レイヤー生成
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'], conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'], conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'], conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout())
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]


In [21]:
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
trainer = Trainer(network, x_train, t_train, x_test, t_test, epochs=20, mini_batch_size=100, optimizer='Adam', optimizer_param={'lr': 0.001}, evaluate_sample_num_per_epoch=1000)

trainer.train()

network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

ValueError: not enough values to unpack (expected 4, got 2)